In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_640cce3148eb4f71af707e493943a70c.txt"
df=pd.read_csv(url,header=None,sep="\t")
df.head()

,0,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
0,33,1,3,2,8,0,5,1,3,7,...,0,0,0,1,0,0,0,0,0,0
1,37,1,2,2,8,1,4,1,4,6,...,0,0,0,1,0,0,0,0,0,0
2,37,1,2,2,8,0,4,2,4,3,...,0,0,0,1,0,0,0,0,0,0
3,9,1,3,3,3,2,3,2,4,5,...,0,0,0,1,0,0,0,0,0,0
4,40,1,4,2,10,1,4,1,4,7,...,0,0,0,1,0,0,0,0,0,0


In [2]:
df.shape

(5822, 86)

In [3]:
X=df.iloc[:,0:85]
X.head()

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,33,1,3,2,8,0,5,1,3,7,...,0,0,0,0,1,0,0,0,0,0
1,37,1,2,2,8,1,4,1,4,6,...,0,0,0,0,1,0,0,0,0,0
2,37,1,2,2,8,0,4,2,4,3,...,0,0,0,0,1,0,0,0,0,0
3,9,1,3,3,3,2,3,2,4,5,...,0,0,0,0,1,0,0,0,0,0
4,40,1,4,2,10,1,4,1,4,7,...,0,0,0,0,1,0,0,0,0,0


In [4]:
y=pd.DataFrame(df.iloc[:,85]).rename(columns={85:"status"})
y.head()

,status
0,0
1,0
2,0
3,0
4,0


In [5]:
df=pd.concat([X,y],axis=1)
df.groupby("status").count().T.head()

status,0,1
0,5474,348
1,5474,348
2,5474,348
3,5474,348
4,5474,348


In [6]:
import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
46,135.632027
67,123.607166
81,65.757955
43,54.664043
42,54.065784
...,...
7,0.094965
40,0.023589
44,0.019971
37,0.005616


In [7]:
X_scores=scores[scores["scores"]>10]
len(X_scores)

40

In [8]:
X=X.iloc[:,X_scores.index]
X.head()

,46,67,81,43,42,58,60,17,41,64,...,23,20,12,21,74,53,16,22,27,8
0,6,1,0,0,3,5,0,7,4,0,...,2,1,1,2,0,0,2,5,6,3
1,0,0,0,2,4,2,0,4,5,2,...,4,0,0,5,0,0,5,0,5,4
2,6,1,0,2,4,2,0,4,3,1,...,2,0,4,7,0,0,5,0,4,4
3,6,1,0,0,4,2,0,2,4,0,...,2,0,2,3,0,0,4,1,4,4
4,0,0,0,0,3,6,0,0,6,0,...,0,4,2,0,0,0,4,0,0,4


In [9]:
df=pd.concat([X,y],axis=1)
X_0=df[df["status"]==0]
X_1=df[df["status"]==1]
y=df[["status"]]
X_1=pd.concat([X_1]*7,axis=0)
df=pd.concat([X_0,X_1],axis=0)
X=df.drop(["status"],axis=1)
y=df[["status"]]

In [11]:
import keras
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout

In [12]:
from sklearn.preprocessing import OneHotEncoder
ienc=OneHotEncoder(handle_unknown="ignore")
y=ienc.fit_transform(y)
y=pd.DataFrame(y.toarray())
y.head()

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [13]:
length=X.shape[1]
num_classes=y.shape[1]
print(length)
print(num_classes)

40
2


In [14]:
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_d046d0e1385449a0b6b3da06c0f834e8.txt"
X_test=pd.read_csv(url,header=None,sep="\t")
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,33,1,4,2,8,0,6,0,3,5,...,0,0,0,0,1,0,0,0,0,0
1,6,1,3,2,2,0,5,0,4,5,...,2,0,0,0,1,0,0,0,0,0
2,39,1,3,3,9,1,4,2,3,5,...,1,0,0,0,1,0,0,0,0,0
3,9,1,2,3,3,2,3,2,4,5,...,0,0,0,0,1,0,0,0,0,0
4,31,1,2,4,7,0,2,0,7,9,...,0,0,0,0,1,0,0,0,0,0


In [15]:
X_test.shape

(4000, 85)

In [16]:
X_test=X_test.iloc[:,X_scores.index]
X_test.head()

,46,67,81,43,42,58,60,17,41,64,...,23,20,12,21,74,53,16,22,27,8
0,0,0,0,1,3,4,0,6,3,1,...,1,1,1,2,0,0,2,6,5,3
1,6,1,0,2,8,4,0,0,6,1,...,0,0,1,4,0,0,4,0,2,4
2,6,1,0,2,5,4,0,4,3,1,...,2,1,2,3,0,0,4,2,2,3
3,5,1,0,2,4,3,0,4,4,1,...,2,1,2,5,0,0,4,1,2,4
4,0,0,0,2,1,1,0,9,3,1,...,4,0,0,2,0,0,0,4,7,7


In [27]:
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_0932b12bc475410081efb122a05966aa.txt"
y_test=pd.read_csv(url,header=None)
y_test.head()

,0
0,0
1,1
2,0
3,0
4,0


In [28]:
y_test=ienc.fit_transform(y_test)
y_test=pd.DataFrame(y_test.toarray())
y_test.head()

,0,1
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [19]:
def classification_model():
    model = Sequential()
    model.add(Dense(length, activation='relu', input_shape=(length,)))
    model.add(Dense(350, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(350,activation="relu"))
    model.add(Dropout(0.4))
    model.add(Dense(350,activation="relu"))
    model.add(Dropout(0.4))
    model.add(Dense(350,activation="relu"))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    opt=Adam(lr=0.003)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [20]:
model=classification_model()
model.fit(X,y,validation_data=(X_test, y_test),epochs=15, verbose=2)
model.evaluate(X_test,y_test,verbose=0)

Train on 7910 samples, validate on 4000 samples
Epoch 1/15
 - 23s - loss: 0.6223 - acc: 0.7020 - val_loss: 0.3957 - val_acc: 0.8355
Epoch 2/15
 - 23s - loss: 0.5350 - acc: 0.7253 - val_loss: 0.3560 - val_acc: 0.8700
Epoch 3/15
 - 22s - loss: 0.5173 - acc: 0.7306 - val_loss: 0.3833 - val_acc: 0.8478
Epoch 4/15
 - 22s - loss: 0.5099 - acc: 0.7343 - val_loss: 0.3682 - val_acc: 0.9205
Epoch 5/15
 - 22s - loss: 0.4952 - acc: 0.7393 - val_loss: 0.3788 - val_acc: 0.8950
Epoch 6/15
 - 22s - loss: 0.4813 - acc: 0.7511 - val_loss: 0.3703 - val_acc: 0.9303
Epoch 7/15
 - 22s - loss: 0.4836 - acc: 0.7526 - val_loss: 0.4214 - val_acc: 0.8043
Epoch 8/15
 - 22s - loss: 0.4754 - acc: 0.7502 - val_loss: 0.4098 - val_acc: 0.8223
Epoch 9/15
 - 22s - loss: 0.4580 - acc: 0.7582 - val_loss: 0.4390 - val_acc: 0.8180
Epoch 10/15
 - 22s - loss: 0.4599 - acc: 0.7625 - val_loss: 0.4676 - val_acc: 0.8003
Epoch 11/15
 - 21s - loss: 0.4507 - acc: 0.7727 - val_loss: 0.4381 - val_acc: 0.8077
Epoch 12/15
 - 21s - loss:

[0.41578835892677307, 0.82075]

In [29]:
from sklearn.metrics import roc_auc_score
y_pred=model.predict(X_test)
print("AUC for test data: ", roc_auc_score(y_test,y_pred))

AUC for test data:  0.7043271056428951


In [30]:
y_pred=pd.DataFrame(ienc.inverse_transform(y_pred))
y_pred.head()

,0
0,0
1,1
2,1
3,0
4,0


In [31]:
y_test=pd.DataFrame(ienc.inverse_transform(y_test))
y_test.head()

,0
0,0
1,1
2,0
3,0
4,0


In [33]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for test data: \n", confusion_matrix(y_test,y_pred))

Confusion matrix for test data: 
 [[3190  572]
 [ 145   93]]


In [48]:
y_pred=pd.DataFrame(model.predict(X_test)).rename(columns={0:"pred_0",1:"pred_1"})
y_pred.head()

,pred_0,pred_1
0,0.715623,0.284377
1,0.399599,0.600402
2,0.450440,0.549560
3,0.665712,0.334288
4,0.995690,0.004310


In [49]:
y_probs1=y_pred[["pred_1"]]

In [50]:
len(y_probs1[y_probs1["pred_1"]>=0.4])

977

In [51]:
y_pred["pred_0"]=np.where((y_pred.pred_0 >=0.4),1,y_pred.pred_0)
y_pred["pred_0"]=np.where((y_pred.pred_0 < 0.4),0,y_pred.pred_0)
y_pred["pred_1"]=np.where((y_pred.pred_1 >=0.4),1,y_pred.pred_1)
y_pred["pred_1"]=np.where((y_pred.pred_1 < 0.4),0,y_pred.pred_1)

In [52]:
y_pred["pred_0"]=np.where((y_pred.pred_0 == y_pred.pred_1),0,y_pred.pred_0)
#y_pred["pred_1"]=np.where((y_pred.pred_0 == y_pred.pred_1),1,y_pred.pred_1)

In [53]:
len00=len(y_pred[y_pred["pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  3023


In [54]:
len11=len(y_pred[y_pred["pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  977


In [55]:
y_pred=pd.DataFrame(ienc.inverse_transform(y_pred))

In [56]:
roc_auc_score(y_test,y_pred)

0.6538348991909363

In [58]:
print("New confusion matrix: \n",confusion_matrix(y_test,y_pred))

New confusion matrix: 
 [[2912  850]
 [ 111  127]]


In [60]:
from sklearn.metrics import accuracy_score
print("New accuracy: ", accuracy_score(y_test,y_pred))

New accuracy:  0.75975


In [61]:
from sklearn.metrics import plot_confusion_matrix

ImportError: cannot import name 'plot_confusion_matrix'